# Part 1: Data Aquisition and Cleaning

### 1. Inspecting the Data Source

The first data source we chose for this project was from a website called "The Numbers" which consolidates helpful statistics for movies. Upon inspecting the website we noticed that there was a 'Report Builder' function on the website that allowed us to determine certain parameters. Since we are interested in analyzing trends from more recent films, we decided to limit our selections to movies released between 2010-2019.

In order to further inspect how the report builder worked so that we could automate the report acquisition, we ran a sample report for the year 2010:

Upon inspecting this report we noticed two important things:

(1) Each report generated will contain only 100 movies. Knowing this, we decided it would be best to run one report for each year we wanted to analyze. This would create a data set of a more suitable size for analysis.

(2) There was a second form that changed the view of the results to include more parameters. We checked off the parameters that we wanted in the form, and then examined the new hyperlink that was generated in the browser to determine it's structure. Our examination revealed that the hyperlink contained a few main components:

#### (a) https://www.the-numbers.com/movies/report/All/All/All/All/All/All/All/All/All/None/None/   
    --> reflects the criteria we outlined in the first form.
#### (b) 2010/2010   
    --> reflects the year we selected in the first form   
#### (c) /None/None/None/None/None/None   
    --> reflects the second part of the criteria we selected in the first form   
#### (d) ?view-order-by=domestic-box-office&show-release-year=On&view-order-direction=desc&show-production-budget=On&show-domestic-box-office=On   
    --> reflects the criteria from the second form, including:   
    --> which criteria to sort the results by (domestic box office gross)   
    --> the financial data that we selected   
#### (e) &show-theatrical-distributor=On&show-genre=On&show-source=On&show-production-method=On&show-creative-type=On   
    --> reflects more criteria from the second form, including:   
    --> Theatrical Distributor i.e. information about studios   
    --> Genre   
    --> Source i.e. where the idea for the film came from   
    --> Production Method i.e. animation versus live action   
    --> Creative Type i.e. provides more context about the genre    

Given this information, we built a web scraper that could obtain the data that we wanted for each year selected.

## 2. Building the Master Data Set

### The Web Scraper

(1) First, we needed to import a few libraries:

In [1]:
import requests # send http requests to the selected webpage
import pandas as pd # read, save, and manipulate data in a dataframe form

(2) Next, we needed to create an iterator using a for loop so that the process of obtaining the information for each year could be automated. This for loop uses a range object to change the value of i based on the year we wish to obtain.   

(3) The URL that we needed was especially long, so for readability we divided it into separate parts.   

(4) Then, the URL was defined using the variables containing each part, with the year inserted as a variable in the appropriate spot, to create a unique URL for the year determined in that iteration.   

(5) The browser information needed for the headers parameter in requests.get() is stored in the (headers) variable.    
(6) requests.get(), with the specified URL as the arguement is stored in a variable (r).

(7) pd.read_html is used to read html objects from the web and return them in pandas data frame form. In this case, the variable (r) is taken as the argument with the method .text.

(8) Use pandas to create a unique CSV with the name of the year.

In [ ]:
# Create an iterator in a for-loop
for i in range(2010,2021,+1): # the range object begins at year 2010 and increases by 1 year, stopping before 2020
    a = 'https://www.the-numbers.com/movies/report/All/All/All/All/All/All/All/All/All/None/None/'
    b = '/'
    c = '/None/None/None/None/None/None'
    d = '?view-order-by=domestic-box-office'
    e = '&show-release-year=On&view-order-direction=desc&show-production-budget=On&show-domestic-box-office=On&show-international-box-office=On&show-worldwide-box-office=On'
    f = '&show-theatrical-distributor=On&show-genre=On&show-source=On&show-production-method=On&show-creative-type=On'
    
    url = a + str(i) + b + str(i) + c + d + e + f # stitching the components of the URL with the year determined by the for loop
    
    print(str(i) + ' ' + url)  # printing the URL to make sure they work!
    print()
    
    header = {
  "User-Agent": "Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/83.0.4103.106 Safari/537.36",
  "X-Requested-With": "XMLHttpRequest"
    } # the header allows the script to act like a browser when accessing the URL 

    r = requests.get(url, headers=header) # utilizing get function in requests in request to navigate to the URL using the specified browsers as the headers parameters

    df = pd.read_html(r.text) #  using pandas to read the HTML obtained above
    
    print(df) # inspecting what was returned
    
    df = df[0] # using view-source in the browser, we noticed that the second form was actually an additional table
    # this line specifies that we only wish to use the first table, which contains our movie data
    
    df_name = 'the-numbers-' + str(i) # creates a unique name for each dataframe produced that can be used to save it as a csv
    
    print(df_name) # checking the name!
    
    df.to_csv(f"{df_name}.csv") # saves the dataframe for a given year as a unique CSV

### Concatenating CSVs

While the code above provides us with the data that we need, cleaning, manipulating, and exploring the data would be much easier with a master CSV that contained data from all of the years.

Each of the CSV files has the same column names, and every entry in each individual data frame contains the same year. The logical way to stitch these data frames together would be along the index, in a vertical fashion, in chronological order. Due to this, we chose the pandas concat() method and specified axis = 0 to indicate that we wish to concatenate the CSVs along the index.   

(1) Next, we need to specify the first CSV as our starting point.

In [ ]:
df = pd.read_csv('the-numbers-2010.csv')

(3) Finally, we created an iterator object that:
- generates the appropriate CSV file names
- store the file name in a variable (f)
- reads the CSV indicated by (f), and stores the resulting pandas data frame object in a variable (y)
- stores the result of concat_f in the variable(df) so that it can be used as the base data frame in the next iteration

In [ ]:
# iterate it over a range
for i in range(2011,2021): # represents the number of tables scraped from tn
    f = 'the-numbers-' + str(i) + '.csv' 
    print(f) # prints the file name to monitor accuracy
    y = pd.read_csv(f) # read and store the csv in a dummy variable
    df = pd.concat([df,y], axis = 0)  # concatenates the data frame to the new csv

(4) Finally, we save the concatenated data frame as our master CSV.

In [ ]:
df.to_csv(f"master_df.csv", index = 'ignore') # saves the dataframe for a given year as a unique CSV

## 3. Exploring the Data

In [ ]:
df = pd.read_csv('master_df.csv') # checking to make sure everything was saved correctly

In [ ]:
df.shape # checking the size of the database, by rows and columns

In [ ]:
df.info() # checking for missing values and examining data types

In [ ]:
df.head() # examining a few rows

## 4. Cleaning the Data

In [ ]:
df = pd.read_csv('master_df.csv')

1) Dropping Unnecessary Extra Index

In [ ]:
df.drop('Unnamed: 0', axis = 1, inplace = True) # removing unneccessary indices

2) Checking for Duplicates

In [ ]:
x = list(df.Title[df.Title.duplicated()]) # checking for duplicates
x

In [ ]:
y = df[df.Title == 'Robin Hood'] # examining duplicates to determine they are actually different movies
y

3) Cleaning up the currency so we can perform numerical operations

In [ ]:
def clean_currency(col):
    """Takes in column as a string, removes 
    American currency punctuation,
    and changes the value to an integer."""
    
    return col.str.replace('$','').str.replace(',','').astype(int)

In [ ]:
for i in [-1,-2,-3,-4]:    # iterating over the four financial columns
    df[df.columns[i]] = clean_currency(df[df.columns[i]])

4) Engineering Features

In [ ]:
df['NetProfits'] = df.WorldwideGross - df.Budget # engineering a NP feature

In [ ]:
df['ROI'] = round((df.NetProfits / df.Budget),0) # engineering ROI ratio feature

5) Dropping some outliers

In [ ]:
df.drop(df[df['ProductionMethod'] == 'Multiple Production Methods'].index, inplace = True) # there was only 1
df.drop(df[df['ProductionMethod'] == 'Stop-Motion Animation'].index, inplace = True) # there was only a handful and they had no ROI

In [ ]:
df = df[df.Year < 2020] # eliminating movies from 2020 due to world events and lack of data

In [ ]:
df.to_csv(r'master_df.csv', index = False) # please don't run this a second time, the master_df.csv is already in the repo